# Social Media Sentimental Analysis

In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
social_media = pd.read_csv(r"C:\Users\Admin\Downloads\archive (11)\Tweets.csv")

In [3]:
social_media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [4]:
social_media.shape

(14640, 15)

In [5]:
social_media.isnull().sum()[social_media.isnull().sum()>0]

negativereason                5462
negativereason_confidence     4118
airline_sentiment_gold       14600
negativereason_gold          14608
tweet_coord                  13621
tweet_location                4733
user_timezone                 4820
dtype: int64

In [6]:
social_media = social_media.iloc[:, [1, 10]]

In [7]:
social_media

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [8]:
social_media.airline_sentiment.value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [9]:
social_media.airline_sentiment.replace({'negative':0, 'neutral':2, 'positive':1 }, inplace = True)

In [10]:
social_media = social_media.rename(columns = {'text': 'X', 'airline_sentiment': 'Y'})

In [11]:
social_media

,Y,X
0,2,@VirginAmerica What @dhepburn said.
1,1,@VirginAmerica plus you've added commercials t...
2,2,@VirginAmerica I didn't today... Must mean I n...
3,0,@VirginAmerica it's really aggressive to blast...
4,0,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,1,@AmericanAir thank you we got on a different f...
14636,0,@AmericanAir leaving over 20 minutes Late Flig...
14637,2,@AmericanAir Please bring American Airlines to...
14638,0,"@AmericanAir you have my money, you change my ..."


In [12]:
import nltk

In [13]:
from nltk.corpus import stopwords 

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [16]:
list1 = list(stopwords.words('english')) 
list1

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [17]:
len(list1)

179

In [18]:
import string

In [19]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
def text_process(social_media):
    nonpuc = [char for char in social_media if char not in string.punctuation]
    nonpuc = "".join(nonpuc)
    return [word for word in nonpuc.split() if word not in list1]

In [21]:
social_media['X'].apply(text_process)

0                    [VirginAmerica, What, dhepburn, said]
1        [VirginAmerica, plus, youve, added, commercial...
2        [VirginAmerica, I, didnt, today, Must, mean, I...
3        [VirginAmerica, really, aggressive, blast, obn...
4                 [VirginAmerica, really, big, bad, thing]
                               ...                        
14635    [AmericanAir, thank, got, different, flight, C...
14636    [AmericanAir, leaving, 20, minutes, Late, Flig...
14637    [AmericanAir, Please, bring, American, Airline...
14638    [AmericanAir, money, change, flight, dont, ans...
14639    [AmericanAir, 8, ppl, need, 2, know, many, sea...
Name: X, Length: 14640, dtype: object

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
bow_transformer =CountVectorizer(analyzer = text_process).fit(social_media["X"]) 

In [24]:
len(bow_transformer.vocabulary_)

19910

In [25]:
bow_transformer.vocabulary_

{'VirginAmerica': 7607,
 'What': 7739,
 'dhepburn': 10419,
 'said': 16967,
 'plus': 15918,
 'youve': 19613,
 'added': 8039,
 'commercials': 9691,
 'experience': 11103,
 'tacky': 18054,
 'I': 4417,
 'didnt': 10434,
 'today': 18398,
 'Must': 5428,
 'mean': 14673,
 'need': 15060,
 'take': 18068,
 'another': 8351,
 'trip': 18580,
 'really': 16398,
 'aggressive': 8129,
 'blast': 8943,
 'obnoxious': 15312,
 'entertainment': 10934,
 'guests': 11980,
 'faces': 11157,
 'amp': 8297,
 'little': 14376,
 'recourse': 16459,
 'big': 8893,
 'bad': 8680,
 'thing': 18255,
 'seriously': 17182,
 'would': 19491,
 'pay': 15694,
 '30': 838,
 'flight': 11390,
 'seats': 17103,
 'playing': 15884,
 'flying': 11493,
 'VA': 7541,
 'yes': 19575,
 'nearly': 15056,
 'every': 11017,
 'time': 18348,
 'fly': 11473,
 'VX': 7559,
 '“ear': 19671,
 'worm”': 19461,
 'won’t': 19427,
 'go': 11825,
 'away': 8626,
 'Really': 6227,
 'missed': 14856,
 'prime': 16115,
 'opportunity': 15447,
 'Men': 5313,
 'Without': 7795,
 'Hats': 

In [26]:
tdm = bow_transformer.transform(social_media["X"])

In [27]:
tdm.shape

(14640, 19910)

In [28]:
from sklearn.model_selection import train_test_split 

In [75]:
train_x , test_x , train_y , test_y = train_test_split(tdm , social_media.Y)

# Naive Bayes

In [76]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [77]:
nb.fit(train_x, train_y)

MultinomialNB()

In [78]:
pred_nb =  nb.predict(test_x)

In [79]:
confusion_matrix(test_y, pred_nb)

array([[2193,   54,   75],
       [ 246,  291,   63],
       [ 380,   65,  293]], dtype=int64)

In [80]:
print(classification_report(test_y, pred_nb))

              precision    recall  f1-score   support

           0       0.78      0.94      0.85      2322
           1       0.71      0.48      0.58       600
           2       0.68      0.40      0.50       738

    accuracy                           0.76      3660
   macro avg       0.72      0.61      0.64      3660
weighted avg       0.75      0.76      0.74      3660



# Logistic Regression

In [81]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [82]:
logreg.fit(train_x, train_y)

LogisticRegression()

In [83]:
pred_logreg = logreg.predict(test_x)

In [84]:
confusion_matrix(test_y, pred_logreg)

array([[2059,   66,  197],
       [ 118,  386,   96],
       [ 237,   58,  443]], dtype=int64)

In [85]:
print(classification_report(test_y, pred_logreg))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2322
           1       0.76      0.64      0.70       600
           2       0.60      0.60      0.60       738

    accuracy                           0.79      3660
   macro avg       0.74      0.71      0.72      3660
weighted avg       0.79      0.79      0.79      3660



In [86]:
# Decision Tree

In [109]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='entropy')

In [110]:
dt.fit(train_x, train_y)

DecisionTreeClassifier(criterion='entropy')

In [111]:
pred_dt = dt.predict(test_x)

In [112]:
confusion_matrix(test_y, pred_dt)

array([[1778,  193,  351],
       [ 134,  346,  120],
       [ 257,   96,  385]], dtype=int64)

In [113]:
print(classification_report(test_y, pred_dt))

              precision    recall  f1-score   support

           0       0.82      0.77      0.79      2322
           1       0.54      0.58      0.56       600
           2       0.45      0.52      0.48       738

    accuracy                           0.69      3660
   macro avg       0.60      0.62      0.61      3660
weighted avg       0.70      0.69      0.69      3660



In [114]:
# Random Forest

In [93]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [94]:
rfc.fit(train_x, train_y)

RandomForestClassifier()

In [115]:
pred_rfc = rfc.predict(test_x)

In [116]:
confusion_matrix(test_y, pred_rfc)

array([[2132,   65,  125],
       [ 206,  298,   96],
       [ 326,   59,  353]], dtype=int64)

In [117]:
print(classification_report(test_y, pred_rfc))

              precision    recall  f1-score   support

           0       0.80      0.92      0.86      2322
           1       0.71      0.50      0.58       600
           2       0.61      0.48      0.54       738

    accuracy                           0.76      3660
   macro avg       0.71      0.63      0.66      3660
weighted avg       0.75      0.76      0.75      3660



- For Sentiment Analysis, firstly i have used 'Natural Language Processing(NLP)' and build a 'Term Document Matrix (TDM)'.
- And after that i have builded a different models to analyze a data, to get intersting insights and to get more accuracy.
- The models that i have builde here are Naive Bayes, Logistic Regression, Decision Tree and Random Forest Models.

#### Here
1. Class 0:
   - Precision: 0.85  -->  This means that 85% of the instances predicted as class 0 are actually class 0.
   - Recall: 0.89  --> This means that 89% of the actual class 0 instances are correctly identified by the model.
   - F1-Score: 0.87  -->  This is the harmonic mean of precision and recall, providing a balance between the two metrics.

2. Class 1:
   - Precision: 0.76  -->  This means that 76% of the instances predicted as class 1 are actually class 1.
   - Recall: 0.64  -->  This means that 64% of the actual class 1 instances are correctly identified by the model.
   - F1-Score: 0.70  -->  This is the harmonic mean of precision and recall.

3. Class 2:
   - Precision: 0.60  -->  This means that 60% of the instances predicted as class 2 are actually class 2.
   - Recall: 0.60  -->  This means that 60% of the actual class 2 instances are correctly identified by the model.
   - F1-Score: 0.60  -->  This is the harmonic mean of precision and recall.

### The Logistic Regression have:

- **Accuracy**: 0.79
  - This means that 79% of all instances are correctly classified by the model.
